# ViT Trainer Notebook — Fire & Smoke Localization

This notebook implements a modular Vision Transformer (ViT) trainer for YOLO-style detection of Fire and Smoke. It follows the provided SRS: configuration UI, YOLO label parsing, dataloaders, ViT backbone options, detection heads, loss/metrics, training loop (AMP + checkpointing), evaluation, and export.

Use the configuration panel below to choose options and click Apply to populate `config` used by subsequent cells.

In [ ]:
# Header / Setup: imports, device selection, reproducibility helpers
import os
import sys
from pathlib import Path
import random
import math
import time
import yaml
from types import SimpleNamespace

# Try imports; if missing, print friendly message. In a notebook you can pip install from a cell if needed.
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from torchvision import transforms as T
except Exception as e:
    raise RuntimeError('PyTorch is required. Please install torch and torchvision in this environment.') from e

try:
    import timm
except Exception:
    timm = None

# Device and seed utility
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# Configuration UI (ipywidgets) -> config dict printed as YAML
# This cell creates an interactive UI; if ipywidgets isn't available, a default config dict is produced.
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except Exception:
    WIDGETS_AVAILABLE = False

default_config = {
    'model': {
        'backbone': 'vit_base_patch16_224',
        'patch_size': 16,
        'pretrained': True,
        'depth': None,
        'num_heads': None,
        'embed_dim': None       
    },
    'train': {
        'batch_size': 8,
        'epochs': 1,
        'lr': 1e-4,
        'accumulate_grad_batches': 1
    },
    'augmentations': {
        'mosaic': False,
        'mixup': False,
    },
    'detection': {
        'head': 'yolo_style',
        'loss': 'giou'
    },
    'misc': {
        'seed': 42,
        'device': str(DEVICE)
    }
}
# start with the default config
config = default_config.copy()

# If a YAML config exists in the workspace, load it and override defaults
cfg_path = Path('config_schema.yaml')
if cfg_path.exists():
    try:
        loaded = yaml.safe_load(cfg_path.read_text())
        if isinstance(loaded, dict):
            config = loaded
            print('Loaded configuration from config_schema.yaml')
            show_config(config)
    except Exception as e:
        print('Failed to load config_schema.yaml:', e)

def show_config(cfg):
    print(yaml.safe_dump(cfg, sort_keys=False))

if WIDGETS_AVAILABLE:
    # Build a compact UI
    backbone = widgets.Dropdown(options=['vit_tiny_patch16_224', 'vit_small_patch16_224', 'vit_base_patch16_224'], value=config.get('model', {}).get('backbone', 'vit_base_patch16_224'), description='Backbone')
    patch = widgets.Dropdown(options=[8,16,32], value=config.get('model', {}).get('patch_size', 16), description='Patch')
    pretrained = widgets.Checkbox(value=config.get('model', {}).get('pretrained', True), description='Pretrained')
    batch_size = widgets.IntSlider(value=config.get('train', {}).get('batch_size', 8), min=1, max=64, step=1, description='Batch')
    epochs = widgets.IntText(value=config.get('train', {}).get('epochs', 1), description='Epochs')
    apply_btn = widgets.Button(description='Apply', button_style='primary')
    out = widgets.Output()

    def on_apply(b):
        cfg = {
            'model': { 'backbone': backbone.value, 'patch_size': patch.value, 'pretrained': pretrained.value },
            'train': {'batch_size': batch_size.value, 'epochs': epochs.value},
            'augmentations': {'mosaic': False, 'mixup': False},
            'detection': {'head': 'yolo_style', 'loss': 'giou'},
            'misc': {'seed': 42, 'device': str(DEVICE)}
        }
        global config
        config = cfg
        with out:
            clear_output()
            print('Applied config:')
            show_config(config)

    apply_btn.on_click(on_apply)
    display(widgets.VBox([widgets.HBox([backbone, patch, pretrained]), widgets.HBox([batch_size, epochs]), apply_btn, out]))
else:
    print('ipywidgets not available; using default or YAML config. You can modify `config` manually.')
    show_config(config)

## Data loader and YOLO label parser
This section provides parsers for YOLO-style labels and a minimal PyTorch Dataset. It also includes a small SyntheticDataset to run a smoke test (small images with boxes).

In [ ]:
from PIL import Image, ImageDraw
import numpy as np

def parse_yolo_label(path, img_w=None, img_h=None, normalized=True):
    # path -> one-line entries: class x_center y_center width height
    boxes = []
    with open(path, 'r') as f:
        for line in f.read().strip().splitlines():
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls = int(parts[0])
            x_c, y_c, w, h = map(float, parts[1:5])
            if normalized and img_w is not None and img_h is not None:
                x_c *= img_w; y_c *= img_h; w *= img_w; h *= img_h
            # convert to x1,y1,x2,y2
            x1 = x_c - w / 2
            y1 = y_c - h / 2
            x2 = x_c + w / 2
            y2 = y_c + h / 2
            boxes.append([cls, x1, y1, x2, y2])
    return boxes

class SyntheticDataset(Dataset):
    """Generates tiny synthetic images with 0-2 rectangular 'smoke/fire' boxes for a smoke test."""
    def __init__(self, length=64, image_size=224, transform=None):
        self.length = length
        self.image_size = image_size
        self.transform = transform

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # blank background
        img = Image.new('RGB', (self.image_size, self.image_size), (20, 20, 30))
        draw = ImageDraw.Draw(img)
        # randomly place 0-2 boxes
        n = random.choice([0,1,1,2])
        boxes = []
        for i in range(n):
            w = random.randint(self.image_size//8, self.image_size//3)
            h = random.randint(self.image_size//8, self.image_size//3)
            x1 = random.randint(0, self.image_size - w)
            y1 = random.randint(0, self.image_size - h)
            x2 = x1 + w
            y2 = y1 + h
            # draw a semi-transparent rectangle to simulate smoke/fire
            color = (200, random.randint(40,120), 30) if random.random() < 0.6 else (120,120,120)
            draw.rectangle([x1, y1, x2, y2], fill=color)
            cls = 0 if color[0] > 150 else 1
            boxes.append([cls, x1, y1, x2, y2])
        img_arr = np.array(img).astype(np.uint8)
        target = {'boxes': np.array([b[1:] for b in boxes], dtype=np.float32), 'labels': np.array([b[0] for b in boxes], dtype=np.int64)}
        if self.transform is not None:
            img_arr = self.transform(Image.fromarray(img_arr))
        else:
            # convert to tensor HWC->CHW and float32 0..1
            img_arr = (torch.from_numpy(img_arr).permute(2,0,1).float() / 255.0)
        return img_arr, target

## Model builder (ViT backbone + simple detection head)
This cell uses `timm` when available to create a ViT backbone and attaches a small detection head that predicts a small grid of boxes+class logits. This is intentionally simple and meant for educational / experimental use in the notebook.

In [ ]:
class SimpleDetectionHead(nn.Module):
    def __init__(self, in_ch, num_classes=2, num_anchors=1):
        super().__init__()
        # predict per-patch: [obj_conf, x, y, w, h, cls1..clsN]
        self.num_classes = num_classes
        out_dim = num_anchors * (5 + num_classes)
        # a single conv layer is enough for demo (treat tokens shaped back to 2D)
        self.head = nn.Conv2d(in_ch, out_dim, kernel_size=1)

    def forward(self, x):
        # x: (B, C, H, W) -> out (B, out_dim, H, W)
        return self.head(x)

class ViTDetector(nn.Module):
    def __init__(self, backbone_name='vit_base_patch16_224', pretrained=True, num_classes=2):
        super().__init__()
        if timm is None:
            raise RuntimeError('timm is required for the ViT backbone. Please install timm.')
        # Create backbone; use features_only to get spatial feature map if supported.
        try:
            # many timm ViTs support features_only; fallback to create_model and adapt.
            self.backbone = timm.create_model(backbone_name, pretrained=pretrained, features_only=True, out_indices=(0,))
            feat_channels = self.backbone.feature_info.channels()[-1] if hasattr(self.backbone, 'feature_info') else self.backbone.num_features
            # features_only returns a list; we'll assume first element is (B, C, H, W)
        except Exception:
            # fallback: create model and use its default representation size
            m = timm.create_model(backbone_name, pretrained=pretrained, features_only=False)
            feat_channels = getattr(m, 'num_features', None) or getattr(m, 'embed_dim', 768)
            # We'll wrap the model to produce a pseudo-spatial feature map by reshaping tokens
            self.backbone = m
        self.det_head = SimpleDetectionHead(feat_channels, num_classes=num_classes)

    def forward(self, x):
        # Try to get a spatial feature map from backbone; handle both features_only and token outputs.
        feats = None
        out = self.backbone(x)
        # timm features_only -> list of tensors; use last
        if isinstance(out, (list, tuple)):
            feats = out[-1]  # (B,C,H,W)
        else:
            # out is (B, N, C) token representation or (B,C) pooled; attempt to reshape tokens to 2D grid
            if out.dim() == 3:
                B, N, C = out.shape
                s = int(math.sqrt(N))
                feats = out.transpose(1,2).reshape(B, C, s, s)
            elif out.dim() == 2:
                # pooled vector; expand spatially (not ideal but ok for demo)
                B, C = out.shape
                feats = out.view(B, C, 1, 1)
        pred = self.det_head(feats)
        return pred

# Example model instantiation (will only run when timm is installed)
def build_model(cfg):
    md = cfg.get('model', {})
    backbone = md.get('backbone', 'vit_base_patch16_224')
    pretrained = md.get('pretrained', True)
    model = ViTDetector(backbone_name=backbone, pretrained=pretrained, num_classes=2)
    return model.to(DEVICE)

## Losses, metrics and utilities
Includes simple IoU and placeholder GIoU (= IoU here) and an example loss combining objectness + bbox + classification terms for demonstration.

In [ ]:
def iou_xyxy(box1, box2, eps=1e-7):
    # boxes: x1,y1,x2,y2
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    a1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    a2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union = a1 + a2 - inter + eps
    return inter / union

def dummy_loss(pred, targets):
    # For the demo smoke test we implement a tiny loss: MSE between predicted map and zero + small cls loss if any target exists.
    return pred.square().mean()

# A simple evaluation stub that returns average number of predicted boxes (for smoke test)
def evaluate_simple(model, dataloader):
    model.eval()
    total = 0
    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs = imgs.to(DEVICE)
            out = model(imgs)
            total += imgs.shape[0]
    return {'samples': total}

## Training loop (AMP, gradient accumulation)
A concise training loop that supports mixed precision and gradient accumulation. It is intentionally small so you can step through and adapt it to the full detection losses in your project.

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def train_one_epoch(model, optimizer, dataloader, epoch, cfg):
    model.train()
    scaler = GradScaler()
    accum_steps = cfg.get('train', {}).get('accumulate_grad_batches', 1)
    total_loss = 0.0
    for i, (imgs, targets) in enumerate(dataloader):
        imgs = imgs.to(DEVICE)
        optimizer.zero_grad() if accum_steps == 1 else None
        with autocast():
            preds = model(imgs)
            loss = dummy_loss(preds, targets)
            loss = loss / accum_steps
        scaler.scale(loss).backward()
        if (i + 1) % accum_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        total_loss += loss.item() * accum_steps
    avg = total_loss / len(dataloader)
    print(f'Epoch {epoch}: avg loss {avg:.6f}')
    return avg

## Export (TorchScript / ONNX)
Small helper cells to export the trained model for inference. These are minimal examples — test them in your environment before using in production.

In [ ]:
def export_torchscript(model, example_input, path='vit_detector.pt'):
    model.eval()
    traced = torch.jit.trace(model.cpu(), example_input.cpu())
    traced.save(path)
    print('Saved TorchScript to', path)

def export_onnx(model, example_input, path='vit_detector.onnx'):
    model.eval()
    torch.onnx.export(model.cpu(), example_input.cpu(), path, opset_version=11)
    print('Saved ONNX to', path)

## Quick smoke test: run one epoch on synthetic data
This cell constructs a small dataset/dataloader, builds the model, and runs one training epoch. It verifies the notebook runs end-to-end for one epoch (AC-1).

In [ ]:
# Smoke test runner
bs = config.get('train', {}).get('batch_size', 8)
epochs = config.get('train', {}).get('epochs', 1)
ds = SyntheticDataset(length=32, image_size=224)
dl = DataLoader(ds, batch_size=bs, shuffle=True, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))

# collate_fn returned lists; convert to batched tensors inside a tiny wrapper
def collate_to_batch(batch):
    imgs, targets = batch
    imgs = torch.stack(imgs).to(DEVICE)
    # keep targets as-is for dummy loss
    return imgs, targets

# recreate dataloader with simple collate
dl = DataLoader(ds, batch_size=bs, shuffle=True, num_workers=0, collate_fn=collate_to_batch)

try:
    model = build_model(config)
except Exception as e:
    print('Model build failed:', e)
    model = None

if model is not None:
    optim = torch.optim.AdamW(model.parameters(), lr=config.get('train', {}).get('lr', 1e-4))
    for ep in range(1, epochs+1):
        train_one_epoch(model, optim, dl, ep, config)
    print('Smoke test completed. Run `evaluate_simple(model, dl)` to get a simple evaluation.')

---
Notes and next steps:
- The notebook is intentionally modular and educational. Replace `dummy_loss` and `evaluate_simple` with production-quality detection losses (GIoU, objectness, classification) and mAP calculation when moving beyond the smoke test.
- The `timm` ViT backbone with `features_only=True` is used when available; some backbones require adapting the token outputs into spatial features (the notebook includes a fallback reshape).
- Add `config_schema.yaml` if you want to persist the selected configuration across runs.